In [2]:
import pandas as pd
import numpy as np
import sys


In [3]:


# Load your data (replace 'your_data.csv' with your actual file)
time_period = "14-00--14-10"
date = "20241015"
data_file_name = f"xnas-itch-{date}_{time_period}"
data_stream = pd.read_csv(f'../../../../../data/stocks_nanosecond/{data_file_name}.csv')



date_column = 'ts_event'

time_delta = 0
batch_results = []
data_stream[date_column] = pd.to_datetime(data_stream[date_column])
previous_event_timestamp = data_stream[date_column].iloc[0]
previous_check_timestamp = data_stream[date_column].iloc[0]




time_start = pd.Timestamp('2024-10-15 14:00:10.00', tz='UTC')
time_end = pd.Timestamp('2024-10-15 14:00:11.00', tz='UTC')


data_stream[date_column] = pd.to_datetime(data_stream[date_column])
# get data between time_start and time_end
data_stream = data_stream[(data_stream[date_column] >= time_start) & (data_stream[date_column] <= time_end)]




In [4]:

print(len(data_stream))

# check ts_event column
print(data_stream[date_column].head())
print(data_stream[date_column].tail())
print(data_stream[date_column].min())
print(data_stream[date_column].max())


10244
74010   2024-10-15 14:00:10.000005390+00:00
74011   2024-10-15 14:00:10.000014516+00:00
74012   2024-10-15 14:00:10.000037823+00:00
74013   2024-10-15 14:00:10.000050011+00:00
74014   2024-10-15 14:00:10.000050508+00:00
Name: ts_event, dtype: datetime64[ns, UTC]
84249   2024-10-15 14:00:10.999595611+00:00
84250   2024-10-15 14:00:10.999595611+00:00
84251   2024-10-15 14:00:10.999595892+00:00
84252   2024-10-15 14:00:10.999595892+00:00
84253   2024-10-15 14:00:10.999628058+00:00
Name: ts_event, dtype: datetime64[ns, UTC]
2024-10-15 14:00:10.000005390+00:00
2024-10-15 14:00:10.999628058+00:00


In [5]:
print(data_stream.groupby("sector").size())

sector
Basic Materials            108
Communication Services    2443
Consumer Cyclical         1925
Consumer Defensive         618
Energy                     492
Financial Services         248
Healthcare                 655
Industrials                 36
Technology                3719
dtype: int64


In [55]:
import pandas as pd
import numpy as np


time_period = "14-00--14-10"
date = "20241015"
data_file_name = f"xnas-itch-{date}_{time_period}"
pre_data_stream = pd.read_csv(f'../../../../../data/stocks_nanosecond/{data_file_name}.csv')

time_start = pd.Timestamp('2024-10-15 14:00:10.30', tz='UTC')

time_end = pd.Timestamp('2024-10-15 14:00:11.00', tz='UTC')


pre_data_stream[date_column] = pd.to_datetime(pre_data_stream[date_column])
# get data between time_start and time_end
pre_data_stream = pre_data_stream[ (pre_data_stream[date_column] <= time_start)]

print("len predatastream: ", len(pre_data_stream))




time_period = "14-00--14-10"
date = "20241015"
data_file_name = f"xnas-itch-{date}_{time_period}"
data_stream = pd.read_csv(f'../../../../../data/stocks_nanosecond/{data_file_name}.csv')



data_stream[date_column] = pd.to_datetime(data_stream[date_column])
# get data between time_start and time_end
data_stream = data_stream[(data_stream[date_column] >= time_start) & (data_stream[date_column] <= time_end)]

print("len of data stream start to end:", len(data_stream))

date_column = 'ts_event'


# Sort by timestamp (if not already sorted)
data_stream = data_stream.sort_values(date_column).reset_index(drop=True)

# Identify indices of technology items
tech_indices = data_stream[data_stream["sector"] == "Technology"].index

# Define a decay function (e.g., linear decay)
def decay_function(index, total_items, decay_rate=0.5):
    """Returns the probability of removing an item based on its position."""
    return decay_rate * (index / total_items)


# # Define a decay function (e.g., exponential decay)
# def decay_function(index, total_items, decay_rate=0.8):
#     """Exponential decay: Probability increases exponentially with position."""
#     return decay_rate * (np.exp(index / total_items) - 1)



# Set decay rate (adjust as needed)
decay_rate = 0.7

# Randomly remove technology items based on the decay function
np.random.seed(42)  # For reproducibility
remove_mask = np.random.rand(len(tech_indices)) < decay_function(tech_indices, len(data_stream), decay_rate)
indices_to_remove = tech_indices[remove_mask]

# Remove the selected items
filtered_data_stream = data_stream.drop(indices_to_remove).reset_index(drop=True)

print("len of filtered_data_stream:", len(filtered_data_stream))


result_data =  pd.concat([pre_data_stream, filtered_data_stream])
print("len of removed_data: ", len(result_data))
# Verify the result
print(result_data.groupby("sector").size())


len predatastream:  76601
len of data stream start to end: 7653
len of filtered_data_stream: 6726
len of removed_data:  83327
sector
Basic Materials             397
Communication Services    13471
Consumer Cyclical         10497
Consumer Defensive         5255
Energy                     3817
Financial Services         3070
Healthcare                 4950
Industrials                 151
Technology                41719
dtype: int64


In [56]:
result_data[date_column].min()

Timestamp('2024-10-15 14:00:00.000258375+0000', tz='UTC')

In [57]:
result_data[date_column].max()

Timestamp('2024-10-15 14:00:10.999628058+0000', tz='UTC')

In [58]:
time_start = pd.Timestamp('2024-10-15 14:00:08.00', tz='UTC')

result_data = result_data[(result_data[date_column] >= time_start)]
print("len of result data: ", len(result_data))

len of result data:  19245


In [59]:
result_data.to_csv(f"linear_decay_filtered_data_decay_rate_{int(decay_rate*10)}.csv", index=False)